In [1]:
from xgboost import XGBRegressor
import bs4
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import metrics
from sklearn.datasets import load_digits
from math import sqrt

In [2]:
df = pd.read_excel('vehicle.xlsx', engine = 'openpyxl')
df = df[:-31]
df.head(10)

,Unnamed: 0,CurbWeight,TopSpeed,Displacement,Power,Torque,vehicle,laptime,Price,Zero
0,1,1626,325.0,4.0,730.0,800.0,AMG GT Black Series,403.62,337800,3.10
1,2,1525,354.0,6.5,770.0,720.0,Lamborghini Aventador SVJ,404.97,432300,2.80
2,3,650,290.0,2.8,455.0,380.0,Radical SR8LM,408.00,NaN,3.30
3,4,1526,328.0,5.2,640.0,600.0,Lamborghini Huracán Performante,412.01,NaN,2.80
4,5,650,270.0,2.6,363.0,271.0,Radical SR8,415.00,NaN,2.70
5,6,1420,312.0,4.0,520.0,469.0,Porsche 911 GT3 RS,416.40,196800,3.30
6,7,1634,345.0,4.6,887.0,1280.0,Porsche 918 Spyder,417.00,803700,2.60
7,8,1525,353.0,6.5,750.0,690.0,Aventador LP750-4 SV,419.73,411900,2.90
8,9,1385,342.0,3.9,720.0,770.0,Ferrari 488 Pista,420.03,304200,2.85
9,10,1527,285.0,8.4,654.0,813.0,Dodge Viper ACR (Mk V),421.30,128100,4.00


In [3]:
Y_data_ = df["laptime"].values
# X_data_ = df[["CurbWeight","TopSpeed", "Power","Torque","Zero"]].values
X_data_ = df[["CurbWeight","Displacement","TopSpeed", "Power","Torque","Zero"]].values
X_train, X_test, Y_train, Y_test = train_test_split(X_data_, Y_data_, test_size=0.2, random_state=1234)

In [4]:
depth_grid = np.arange(1,21)                                    # 하이퍼파라미터의 범위 설정
n_estimators=np.arange(50,200)
min_samples_leaf_grid = np.arange(2,31,2)
max_leaf_nodes_grid = np.arange(2,51,2)
parameters = {'max_depth':depth_grid, 'min_samples_leaf':min_samples_leaf_grid, 'max_leaf_nodes':max_leaf_nodes_grid }

In [5]:
gridCV = GridSearchCV(XGBRegressor(), parameters, cv=10, n_jobs = -1)      # XGBoostRegressor함수 입력
gridCV.fit(X_train, Y_train)                                                # 데이터의 최적화된 하이퍼파라미터의 찾기
best_depth = gridCV.best_params_['max_depth'] 
best_min_samples_leaf = gridCV.best_params_['min_samples_leaf']
best_max_leaf_nodes = gridCV.best_params_['max_leaf_nodes']

[10:39:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { max_leaf_nodes, min_samples_leaf } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [6]:
XGBR_best = XGBRegressor(max_depth=best_depth,min_samples_leaf=best_min_samples_leaf,max_leaf_nodes=best_max_leaf_nodes)
XGBR_best.fit(X_train, Y_train)                                             # 최적화된 하이퍼파라미터를 입력하고 훈련
Y_pred = XGBR_best.predict(X_test)                                          # 예측모델의 Test데이터를 이용하여 RMSE 출력
print( "Tree best RMSE : " + str(np.round(np.sqrt(metrics.mean_squared_error(Y_test,Y_pred)),3)))

[10:39:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { max_leaf_nodes, min_samples_leaf } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Tree best RMSE : 11.451


In [7]:
best_depth = gridCV.best_params_['max_depth'] 
best_min_samples_leaf = gridCV.best_params_['min_samples_leaf']
best_max_leaf_nodes = gridCV.best_params_['max_leaf_nodes']
print(best_depth, best_min_samples_leaf, best_max_leaf_nodes)

4 2 2
